In [ ]:
from supervised.data import TargetGaussianDataset
import torch
from torch.utils.data import DataLoader
from supervised.utils import split_state_information
import numpy as np

import matplotlib.pyplot as plt

num_joints = 2
# checkpoint = torch.load("")

data = DataLoader(TargetGaussianDataset(state_file=f"./datasets/{num_joints}/test/state_IK_random_start.csv", std=0.5), batch_size=1)

state_angles_list = []
actions = []
for x, y in data:
    target_position, current_position, state_angles = split_state_information(x)
    state_angles_list.append(state_angles)
    actions.append(y)

state_angles_list = torch.stack(state_angles_list).detach().numpy().squeeze()
actions = torch.cat(actions).detach().numpy().squeeze()

# for joint_idx in range(num_joints):
#     plt.hist(state_angles_list[:, joint_idx])

fig, axs = plt.subplots(3, 1)
axs[0].hist2d(state_angles_list[:, 0], state_angles_list[:, 1], bins=20)

axs[1].hist2d(actions[:, 0], actions[:, 1], bins=20)

abs_actions = state_angles_list + actions
axs[2].hist2d(abs_actions[:, 0], abs_actions[:, 1])

plt.show()


In [ ]:
from vae.utils.post_processing import PostProcessor
from supervised.model import Regressor

from vae.utils.fk import forward_kinematics


data = DataLoader(TargetGaussianDataset(state_file=f"./datasets/{num_joints}/val/state_IK_random_start.csv", std=0.5), batch_size=50)

checkpoint = torch.load("results/supervised/test/PointDistanceLoss/2_1684662875/model_1000_val_loss_0.1297.pt")

model = Regressor(4 + num_joints, num_joints, learning_rate=0, post_processor=PostProcessor(enabled=False))

model.load_state_dict(checkpoint["model_state_dict"])

actions = []
dists = []
for x, y in data:
    x_hat = model.forward(x)
    target_position, current_position, state_angles = split_state_information(x)
    dists.append(torch.linalg.norm(forward_kinematics(x_hat + state_angles)[:, -1] - target_position, dim=1).mean()) 
    actions.append(x_hat)

actions = torch.cat(actions).detach().numpy().squeeze()

print(torch.tensor(dists).mean())

# heatmap, x_edges, y_edges = np.histogram2d(actions[:, 0], actions[:, 1])
plt.hist2d(actions[:, 0], actions[:, 1])
plt.imshow(heatmap)